<a href="https://colab.research.google.com/github/DilshodbekMX/GoogleColab/blob/main/YOLOv8n%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
from torch.nn.modules.upsampling import Upsample
import torch
import torch.nn as nn
import torch.nn.functional as F

In [131]:
class Bottleneck(nn.Module):
    """Standard bottleneck."""

    def __init__(self, c1, c2, shortcut=True, g=1, k=(3, 3), e=0.5):
        """Initializes a bottleneck module with given input/output channels, shortcut option, group, kernels, and
        expansion.
        """
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, k[0], 1)
        self.cv2 = Conv(c_, c2, k[1], 1, g=g)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        """'forward()' applies the YOLO FPN to input data."""
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))

In [132]:
class C2f(nn.Module):
    """Faster Implementation of CSP Bottleneck with 2 convolutions."""

    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):
        """Initialize CSP bottleneck layer with two convolutions with arguments ch_in, ch_out, number, shortcut, groups,
        expansion.
        """
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, 2 * self.c, 1, 1,act=True)
        self.cv2 = Conv((2 + n) * self.c, c2, 1,act=True)  # optional act=FReLU(c2)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=0.5) for _ in range(n))

    def forward(self, x):
        """Forward pass through C2f layer."""
        y = list(self.cv1(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))

    def forward_split(self, x):
        """Forward pass using split() instead of chunk()."""
        y = list(self.cv1(x).split((self.c, self.c), 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))


In [133]:
class SPPF(nn.Module):
    """Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5 by Glenn Jocher."""

    def __init__(self, c1, c2, k=5):
        """
        Initializes the SPPF layer with given input/output channels and kernel size.

        This module is equivalent to SPP(k=(5, 9, 13)).
        """
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)

    def forward(self, x):
        """Forward pass through Ghost Convolution block."""
        x = self.cv1(x)
        y1 = self.m(x)
        y2 = self.m(y1)
        return self.cv2(torch.cat((x, y1, y2, self.m(y2)), 1))


In [134]:
def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p


class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""

    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        """Perform transposed convolution of 2D data."""
        return self.act(self.conv(x))


class Concat(nn.Module):
    """Concatenate a list of tensors along dimension."""

    def __init__(self, dimension=1):
        """Concatenates a list of tensors along a specified dimension."""
        super().__init__()
        self.d = dimension

    def forward(self, x):
        """Forward pass for the YOLOv8 mask Proto module."""
        return torch.cat(x, self.d)



In [135]:
if __name__ =="__main__":
    x = torch.randn(1,3,640,640)
    cv1 = Conv(3,16,3,2)
    cv2 = Conv(16,32,3,2)
    c2f1 = C2f(32, 32, 1, True)
    cv3 = Conv(32, 64, 3, 2)
    c2f2 = C2f(64, 64, 2, True)
    cv4 = Conv(64, 128, 3, 2)
    c2f3 = C2f(128, 128, 2, True)
    cv5 = Conv(128, 256, 3, 2)
    c2f4 = C2f(256, 256, 2, True)
    sppf = SPPF(256,256,5)

    ups1 = Upsample(None, 2, 'nearest')
    con1 = Concat(1)
    c2f5 = C2f(384, 128, 1, False)
    ups2 = Upsample(None, 2, 'nearest')
    con2 = Concat(1)
    c2f6 = C2f(192, 64, 1, False)

    cv6 = Conv(64, 64, 3, 2)
    con3 = Concat(1)
    c2f7 = C2f(192, 128, 1, False)
    cv7 = Conv(128, 128, 3, 2)
    con4 = Concat(1)
    c2f8 = C2f(384, 256, 1, False)

    x = cv1(x)
    print(f"cv1: {x.shape}")
    x = cv2(x)
    print(f"cv2: {x.shape}")
    x = c2f1(x)
    print(f"c2f1: {x.shape}")
    x = cv3(x)
    print(f"cv3: {x.shape}")
    x = c2f2(x)
    b1 = x
    print(f"c2f2: {x.shape}")
    x = cv4(x)
    print(f"cv4: {x.shape}")
    x = c2f3(x)
    b2 = x
    print(f"c2f3: {x.shape}")
    x = cv5(x)
    print(f"cv5: {x.shape}")
    x = c2f4(x)
    print(f"c2f4: {x.shape}")
    x = sppf(x)
    print(f"sppf: {x.shape}")
    b3 = x

    print(" ")
    x = ups1(x)
    print(f"ups1: {x.shape}")
    x = con1([x,b2])
    print(f"con1: {x.shape}")
    x = c2f5(x)
    h1 = x
    print(f"c2f5: {x.shape}")
    x = ups2(x)
    print(f"ups2: {x.shape}")
    x = con2([x,b1])
    print(f"con2: {x.shape}")
    x = c2f6(x)
    print(f"c2f6: {x.shape}")

    x = cv6(x)
    print(f"cv6: {x.shape}")
    x = con3([x,h1])
    print(f"con3: {x.shape}")
    x = c2f7(x)
    print(f"c2f7: {x.shape}")

    x = cv7(x)
    print(f"cv7: {x.shape}")
    x = con4([x,b3])
    print(f"con4: {x.shape}")
    x = c2f8(x)
    print(f"c2f8: {x.shape}")

cv1: torch.Size([1, 16, 320, 320])
cv2: torch.Size([1, 32, 160, 160])
c2f1: torch.Size([1, 32, 160, 160])
cv3: torch.Size([1, 64, 80, 80])
c2f2: torch.Size([1, 64, 80, 80])
cv4: torch.Size([1, 128, 40, 40])
c2f3: torch.Size([1, 128, 40, 40])
cv5: torch.Size([1, 256, 20, 20])
c2f4: torch.Size([1, 256, 20, 20])
sppf: torch.Size([1, 256, 20, 20])
 
ups1: torch.Size([1, 256, 40, 40])
con1: torch.Size([1, 384, 40, 40])
c2f5: torch.Size([1, 128, 40, 40])
ups2: torch.Size([1, 128, 80, 80])
con2: torch.Size([1, 192, 80, 80])
c2f6: torch.Size([1, 64, 80, 80])
cv6: torch.Size([1, 64, 40, 40])
con3: torch.Size([1, 192, 40, 40])
c2f7: torch.Size([1, 128, 40, 40])
cv7: torch.Size([1, 128, 20, 20])
con4: torch.Size([1, 384, 20, 20])
c2f8: torch.Size([1, 256, 20, 20])
